In [1]:
import numpy as np


import matplotlib.pyplot as plt
import matplotlib.text as text

## Linear regression

$$ y_i = x_{ij} w_j + b$$

$$ y_i = x_{ij} w_j, \quad x_{i,-1}=1,\quad b=w_{-1} $$

In [2]:
def linear(x,w):
    return x @ w

Generate a random feature vector $\mathbf{x}$ witch 10000 samples and three feature 
such that first feature is drawn from N(0,1), second feature from  U(,1) and third from N(1,2).

In [3]:
x = np.stack((np.random.normal(0, 1, (1000)), 
              np.random.uniform(0, 1, (1000)), 
              np.random.normal(1,2, (1000))), axis=1)
x.shape

(1000, 3)

N(mu,sigma) denotes normal distribution with mean mu and standard deviation sigma. You can use ``numpy.random.normal`` and ``numpy.random.uniform`` functions.

Using $\mathbf{x}$ and weights w = [0.2, 0.5,-0.25,1.0] generate output $\mathbf{y}$ assuming a $N(0,0.1)$ noise $\mathbf{\epsilon}$. 

In [4]:
w = np.array((0.2, 0.5, -0.25, 1.))
ones = np.ones((x.shape[0], 1))
x = np.concatenate((x,ones), axis = 1)
noise = np.random.normal(0, 0.1, x.shape[0])
y = linear(x,w)
y = y + noise
#set random weights
w = np.random.normal(0,1000,(4))
print(w)

[  122.13588729 -2051.10212788  -272.19292822  -101.13199091]


$$ y_i = x_{ij} w_j+\epsilon_i, \quad x_{i,-1}=1,\quad b=w_{-1} $$

#### Loss

$$ \frac{1}{2}\frac{1}{N}\sum_{i=0}^{N-1} (y_i -  x_{ij} w_j  )^2$$

In [5]:
def getLoss(y, x, w):
    loss = np.square(y - linear(x,w))
    loss = np.sum(loss) / (2*y.shape[0])
    return loss


## Gradient descent 

### Problem 1 

Find the gradient of the loss function with respect to weights.

Write gradient function ``grad(y,x,w)``.

In [6]:
def grad(y, x, w):
    diff = (x @ w - y)
    return np.dot(x.T, diff) / x.shape[0]
#     return (w - (alpha/x.shape[0]) * tmp)
gradient = grad(y, x, w)
print(gradient)

[  143.03142289  -872.68466279 -2482.11051852 -1387.40437993]


### Problem 2

Implement gradient descent for linear regression.

In [7]:
alpha = 0.3

def gradientDescent(y, x, w, maxIterations = 500, tolerance=0.01):
    for i in range(maxIterations):
        loss = getLoss(y, x, w)
        if loss < tolerance:
            maxIterations = i
            break
        gradient = grad(y, x, w)
        w = w - alpha*gradient
        if i % 10 == 0:
            print("i="+str(i)+", loss="+str(loss))

    print("finished gradient descent on iteration " + str(maxIterations))
    print("with loss equal " + str(loss))
    return w
        
gradientDescent(y, x, w)

i=0, loss=1312266.4281548972
i=10, loss=67835.92447028619
i=20, loss=45603.79038656082
i=30, loss=30846.49466929855
i=40, loss=20864.784866291964
i=50, loss=14113.087192412471
i=60, loss=9546.1919720036
i=70, loss=6457.112311988214
i=80, loss=4367.637365578803
i=90, loss=2954.3019931750837
i=100, loss=1998.3121267638426
i=110, loss=1351.6739384570963
i=120, loss=914.2833747408512
i=130, loss=618.4293582483045
i=140, loss=418.3116457602121
i=150, loss=282.9506345214962
i=160, loss=191.39150596780593
i=170, loss=129.46027521830115
i=180, loss=87.56956041208414
i=190, loss=59.23438984081561
i=200, loss=40.068283580348364
i=210, loss=27.10419613066849
i=220, loss=18.335197299142145
i=230, loss=12.403785558379052
i=240, loss=8.391737442694852
i=250, loss=5.6779602196531345
i=260, loss=3.842342449390154
i=270, loss=2.6007178348506588
i=280, loss=1.7608743181856992
i=290, loss=1.1927983278866772
i=300, loss=0.8085477807898732
i=310, loss=0.5486380407804875
i=320, loss=0.3728332866633048
i=330

array([ 0.19867092,  0.17312994, -0.24973335,  1.17387445])

### Problem 3

Implement stochastic gradient descent (SGD).

In [8]:
def sgd(y, x, w, maxIterations = 500, tolerance=0.01, batchSize = 10):
    for i in range(maxIterations):
        loss = getLoss(y, x, w)
        if loss < tolerance:
            maxIterations = i
            break
        randomIndices = np.random.randint(1000, size=(batchSize))
        selectedX = x[randomIndices]
        selectedY = y[randomIndices]
        gradient = grad(selectedY, selectedX, w)
        w = w - alpha*gradient
        if i % 10 == 0:
            print("i="+str(i)+", loss="+str(loss))

    print("finished stochastic gradient descent on iteration " + str(maxIterations))
    print("with loss equal " + str(loss))
    return w
sgd(y, x, w)

i=0, loss=1312266.4281548972
i=10, loss=168307.3226750543
i=20, loss=51208.63851445202
i=30, loss=34872.303851870965
i=40, loss=25321.61575590273
i=50, loss=15834.375795431066
i=60, loss=12432.856309043458
i=70, loss=17468.001216947534
i=80, loss=15963.719787123322
i=90, loss=7467.246713554412
i=100, loss=2506.5141574037525
i=110, loss=1550.5866160054206
i=120, loss=1042.3605519160853
i=130, loss=689.1586547404727
i=140, loss=1035.4476679108295
i=150, loss=679.8665317435531
i=160, loss=226.90333055716368
i=170, loss=283.401979829168
i=180, loss=99.94022618576761
i=190, loss=66.05646168383538
i=200, loss=47.92871342633085
i=210, loss=34.432633434397765
i=220, loss=26.101764330770028
i=230, loss=136.93389363718285
i=240, loss=5.654770843269113
i=250, loss=8.812139400334784
i=260, loss=2.5449690810958416
i=270, loss=1.9418411610512245
i=280, loss=1.1694025106957628
i=290, loss=0.9708191573837514
i=300, loss=0.5893909776070682
i=310, loss=0.4009049271279307
i=320, loss=0.26823729954153863


array([ 0.21622965,  0.23483976, -0.24903316,  1.0922328 ])

In [9]:
#set random weights
w = np.random.normal(0,1000,(4))
print("SGD takes: ")
%time tSGD = sgd(y, x, w)
#set random weights
w = np.random.normal(0,1000,(4))
print("gradient descent takes: ")
%time tGD = gradientDescent(y, x, w)

SGD takes: 
i=0, loss=1774508.9541529212
i=10, loss=11323.975449907388
i=20, loss=4824.496973278289
i=30, loss=2987.113102007137
i=40, loss=3370.72455418032
i=50, loss=1346.2393655775008
i=60, loss=1071.9215565907543
i=70, loss=1217.1014062484294
i=80, loss=868.5892651844157
i=90, loss=282.92305406897077
i=100, loss=196.5308352616469
i=110, loss=175.0897587423172
i=120, loss=108.82145176174073
i=130, loss=66.73697168440447
i=140, loss=69.86330556692812
i=150, loss=28.85455056524625
i=160, loss=37.74475724329161
i=170, loss=13.86319448301725
i=180, loss=10.822622404521365
i=190, loss=7.612568718088056
i=200, loss=7.312812972973595
i=210, loss=3.2251331330489164
i=220, loss=2.530378219911061
i=230, loss=1.720699224931459
i=240, loss=1.065385573208324
i=250, loss=0.9025151600485497
i=260, loss=0.5654545535066945
i=270, loss=1.861644765256073
i=280, loss=0.2632660108974288
i=290, loss=0.4608242451705873
i=300, loss=0.1132977054130163
i=310, loss=0.1190963440830426
i=320, loss=0.06507818243

### Problem 4

Implement SGD using pytorch. Start by just rewritting Problem 3 to use torch Tensors instead of numpy arrays. 

To convert frrom numpy arrays to torch tensors you can use ``torch.from_numpy()`` function. 

### Problem 5 

Implement GD using pytorch automatic differentiation.

To this end the variable with respect to which the gradient will be calculated, ``t_w`` in this case, must have attribute
``requires_grad`` set to ``True`` (``t_w.require_grad=True``).

The torch will automatically track any expression containing ``t_w`` and store its computational graph. The method ``backward()`` can be run on the final expression to back propagate the gradient e.g. ``loss.backward()``. Then the gradient is accesible as ``t_w.grad``.